# Sequential pattern

In [1]:
from os import environ
environ.setdefault("HF_HOME", "/data/hf_models/")

'/data/hf_models/'

In [ ]:
import sys
sys.path.append("../../src/")

In [ ]:
from agent_design_pattern.agent import AgentMessage, LLMChain, BaseAgent
from agent_design_pattern.orchestration import SequentialAgent
from chain import CasualSingleTurnChain

## Define the agent

Create a simple agent for demonstration

In [4]:
class FunctionImplementationAgent(BaseAgent):
    def __init__(self, llm_chain: LLMChain, state_change_callback = None, name = None, **kwargs):
        super().__init__(state_change_callback, name, **kwargs)
        self.llm_chain = llm_chain

    def execute(self, message: AgentMessage, **kwargs):
        return self.llm_chain.invoke(message, **kwargs)

In [5]:
code_system_prompt = """You are a helpful coding assistant.
You task is to write a python function and return the implementation of the function.
Just the implementation, DO NOT comment anything else"""
code_user_prompt = "{query}"
code_chain = CasualSingleTurnChain("swiss-ai/Apertus-8B-Instruct-2509", code_system_prompt, code_user_prompt)

doc_system_prompt = """You are a great developer avocate.
You task is to write a document and in-code explain for the function.
Just the comment and explanation, DO NOT change the logic of the provided function.
The response should contain function with docstring explanation. And DO NOT contain explanation outside of the code"""
doc_user_prompt = """Input query: {query}

Function implementation: {response}
"""
doc_chain = CasualSingleTurnChain("HuggingFaceTB/SmolLM3-3B", doc_system_prompt, doc_user_prompt)

review_system_prompt = """You are a excellent code reviewer and refactor.
Given a function implementation and it explanation, your task is to review and code and correct if contains any mistake.
Some note:
- For the implementation, check if the orignal query and suggested implementation are match.
- Is there any syntax error in the code.
- For the explanation, verify if the docstring follows Google style docstring.
- In the docstring, make sure to have an example to call the function.

Make sure the final output only contain code, inline code comment and docstring, nothing else."""
review_user_prompt = """Input query: {query}

Function implementation: {response}
"""
review_chain = CasualSingleTurnChain("ibm-granite/granite-4.0-h-1b", review_system_prompt, review_user_prompt)

def state_callback(message: str):
    print(message)

code_agent = FunctionImplementationAgent(code_chain, state_change_callback=state_callback, name="Code Agent")
doc_agent = FunctionImplementationAgent(doc_chain, state_change_callback=state_callback, name="Docstring Agent")
review_agent = FunctionImplementationAgent(review_chain, state_change_callback=state_callback, name="Review Agent")
agent = SequentialAgent([code_agent, doc_agent, review_agent], state_change_callback=state_callback, name="Sequential Agent")

Fetching 4 files: 100%|██████████| 4/4 [06:11<00:00, 92.76s/it] 
CUDA-fused xIELU not available (No module named 'xielu') – falling back to a Python version.
For CUDA xIELU (experimental), `pip install git+https://github.com/nickjbrowning/XIELU`
Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.31s/it]
The fast path is not available because one of `(selective_state_update, causal_conv1d_fn, causal_conv1d_update)` is None. Falling back to the naive implementation. To install follow https://github.com/state-spaces/mamba/#installation and https://github.com/Dao-AILab/causal-conv1d


In [6]:
query = "Write a python function to find the biggest but not exceed the given integer number. The found number must be a number in Fibonacci array."
message = agent.execute(AgentMessage(query=query), max_new_tokens=16384)

print(message)
print(message.response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


running agent Code Agent
running agent Docstring Agent
running agent Review Agent
idle
query='Write a python function to find the biggest but not exceed the given integer number. The found number must be a number in Fibonacci array.' origin='Sequential Agent' response='```' responses=None context=None execution_result='success' error_message=None
```
